In [160]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [194]:
def request_htmL(ano):
    # extraindo html
    url = f'https://www.basketball-reference.com/teams/CLE/{ano}_games.html'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup 

In [195]:
# definindo as colunas
columns = ['G','Date','Start (ET)','#','##','Out Home','Opponent', 'W\L', 'OT','Tm','Opp','W','L','Streak','Notes']

In [229]:
# Criando dataframes
df_dados = pd.DataFrame(columns=columns)
df_StatsNBA = pd.DataFrame(columns=columns)

count_anos = 1982


In [233]:

rows = []
count_index = 1


while count_anos <= 1983:


    # Extraindo os jogos do html
    html = request_htmL(count_anos)
    jogos = html.find_all('tr')

    for jogo in jogos[1:]:


        # Verificando se a coluna [G] é um numero
        if jogo.findAll('th')[0].get_text().isnumeric():
            
            # Armazenando o numero do jogo e fazendo append na lista de linhas 
            numGame = jogo.findAll('th')[0].get_text()
            rows.append(numGame)

            # for para percorrer a tabela que contem as estatisticas e armazenando na lista de linhas
            if count_anos < 1986:
                
                for infoJogo in jogo.findAll('td'):
                
                    if count_index == 2:
                       
                        rows.append(np.NaN)
                        rows.append(infoJogo.get_text())
                            
                        count_index+=1
                
                    else:
                        rows.append(infoJogo.get_text())
                        count_index+=1
            else:
            
                for infoJogo in jogo.findAll('td'):
                    rows.append(infoJogo.get_text())

            count_index = 1 

            # adicionando os dados no datafreme
            df_dados[columns] = [rows]
            df_StatsNBA = pd.concat([df_StatsNBA, df_dados])
            rows.clear()

    
    print(count_anos)       
    count_anos+=1
    sleep(randint(1, 20))

               



1982


ValueError: Columns must be same length as key

In [ ]:
df_StatsNBA.drop(['##', '#', 'Notes'], axis=1, inplace=True)

In [231]:
df_StatsNBA

,G,Date,Start (ET),#,##,Out Home,Opponent,W\L,OT,Tm,Opp,W,L,Streak,Notes


In [225]:
h = request_htmL(1983)
js = html.find_all('tr')[1:]

In [228]:
j = js[0]
j.get_text()

'1Fri, Oct 29, 1982Box ScoreBoston CelticsL9310401L 1'

In [ ]:
'''
1
Fri, Oct 29, 1982
Box Score
Boston Celtics
L
93
104
0
1
L
 
1
'''

In [232]:
rows

['1',
 'Fri, Oct 30, 1981',
 '',
 'Box Score',
 '@',
 'Philadelphia 76ers',
 'L',
 '',
 '104',
 '128',
 '0',
 '1',
 'L 1',
 '']